## Cleaning Resume and Job Posts Data for use by Job Recommender System 

### Datasets
* https://www.kaggle.com/maitrip/resumes; 1219 resumes; ID, category, text description
* https://www.kaggle.com/samdeeplearning/deepnlp; 125 resumes; may drop this one
* https://www.kaggle.com/madhab/jobposts/data; 19,000 job posts from 2004-2015.




Imports

In [148]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import nltk
import spacy
import string

First, start with resume dataset

In [149]:
#resume_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/resume_dataset_maitrip.csv'
# that's my home path

lightsail_resume_path = '/home/ec2-user/NLP_projects/job_recommender_project/data/resume_dataset_maitrip.csv'
local_resume_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/resume_dataset_maitrip.csv'


resume_path = local_resume_path



In [150]:
df = pd.read_csv(resume_path)
df.head()

,ID,Category,Resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."


In [151]:
#what does one resume look like
df.Resume[2]

"b'Anthony Brown\\nHR Assistant\\nAREAS OF EXPERTISE\\n\\nPERSONAL SUMMARY\\n\\nHR processes & systems\\n\\nA competent and organised individual who is able to work as part of a team and\\nmanage several priorities at any one time. Anthony has a positive attitude,\\nstrong work ethic, and a keen desire to learn and grow within a firm. He\\npossesses superb communications skills, and always treats people with respect\\nand according to their individual needs. As a dedicated professional he fully\\nunderstands the importance of the HR department to any organisation, and\\ntherefore aims to make any office he works in as effective and efficient as\\npossible. He has extensive experience of working in commercially focussed\\norganisations, and fully understands the pressures of achieving targets and\\naccurately assessing job applicants according to their ability.\\n\\nContract document\\ngeneration\\nAccepting resignations\\nBusiness Administration\\nNote taking\\n\\nRight now he would li

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

## Text Preprocessing

https://www.kaggle.com/sudalairajkumar/getting-started-with-text-preprocessing

In [152]:
df["dirty_resume"] = df["Resume"]
df.drop(["Resume"], axis=1, inplace=True)
df.head()

,ID,Category,dirty_resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."


In [153]:
#remove unicode
def unicodetoascii(text):

    TEXT = (text.
    		replace('\\xe2\\x80\\x99', "'").
            replace('\\xc3\\xa9', 'e').
            replace('\\xe2\\x80\\x90', '-').
            replace('\\xe2\\x80\\x91', '-').
            replace('\\xe2\\x80\\x92', '-').
            replace('\\xe2\\x80\\x93', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x94', '-').
            replace('\\xe2\\x80\\x98', "'").
            replace('\\xe2\\x80\\x9b', "'").
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9c', '"').
            replace('\\xe2\\x80\\x9d', '"').
            replace('\\xe2\\x80\\x9e', '"').
            replace('\\xe2\\x80\\x9f', '"').
            replace('\\xe2\\x80\\xa6', '...').
            replace('\\xe2\\x80\\xb2', "'").
            replace('\\xe2\\x80\\xb3', "'").
            replace('\\xe2\\x80\\xb4', "'").
            replace('\\xe2\\x80\\xb5', "'").
            replace('\\xe2\\x80\\xb6', "'").
            replace('\\xe2\\x80\\xb7', "'").
            replace('\\xe2\\x81\\xba', "+").
            replace('\\xe2\\x81\\xbb', "-").
            replace('\\xe2\\x81\\xbc', "=").
            replace('\\xe2\\x81\\xbd', "(").
            replace('\\xe2\\x81\\xbe', ")").
            replace('\\xef\\x81\\xb3', " ").
            replace('\\n\\xef\\x82\\xb7', ' ').
            replace('xefxxb', ' ').
            replace('\\n', ' ').
            replace('\n', ' ').
            replace('\x0c', ' ').
            replace('xexxa', ' ').
            replace('xefxxae', ' ').
            replace('xcxa', ' ').
            replace('xefxxa', ' ').
            replace('xefxx', ' ')
             )
    return TEXT

In [154]:
#remove punctuation
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [155]:
#remove stopwords
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/richardkuzma/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [156]:
#remove URLs
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

In [157]:
#remove HTML
def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

In [158]:
#remove digits
DIGITS_TO_REMOVE = "0123456789"
def remove_digits(text):
    return text.translate(str.maketrans('', '', DIGITS_TO_REMOVE))

In [159]:
#remove endlines
END_LINES = '\r\n'
def remove_endlines(text):
    """custom function to remove \r\n for end of line"""
    return text.replace(END_LINES, ' ')

In [160]:
#remove hyphens
#use this before remove_punctuation, becuase words like 'detail-oriented' get mushed
#together by remove_punctuation, which replaces '-' with ''
def remove_hyphens(text):
    """custom function to remove \r\n for end of line"""
    return text.replace('-', ' ')

In [161]:
# df["resume"] = df["resume"].astype(str).str.lower()
# df['resume'] = df['resume'].apply(remove_urls)
# df['resume'] = df['resume'].apply(remove_html)
# df['resume'] = df["resume"].apply(unicodetoascii)
# df['resume'] = df['resume'].apply(remove_hyphens)
# df['resume'] = df['resume'].apply(remove_endlines)
# df['resume'] = df['resume'].apply(remove_punctuation)
# df['resume'] = df['resume'].apply(remove_stopwords)
# df['resume'] = df['resume'].apply(remove_digits)
# df['resume'] = df['resume'].str.slice_replace(0,1, '') #remove the b at the beginning

In [162]:
# one cleaning method to rule them all
def full_clean(dataframe, clean_series, dirty_series):
    dataframe[clean_series] = dataframe[dirty_series]
    
    dataframe[clean_series] = dataframe[clean_series].apply(unicodetoascii)
    dataframe[clean_series] = dataframe[clean_series].astype(str).str.lower()
    dataframe[clean_series] = dataframe[clean_series].apply(remove_urls)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_html)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_hyphens)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_endlines)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_punctuation)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_stopwords)
    dataframe[clean_series] = dataframe[clean_series].apply(remove_digits)
    
    return dataframe

In [163]:
df = full_clean(df, 'resume', 'dirty_resume')

In [164]:
df.head()

,ID,Category,dirty_resume,resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",bjohn h smith phr po box callahan fl info...
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,bname surname address mobile noemail personal ...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,banthony brown hr assistant areas expertise pe...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,b satheesh email id career objective pursue gr...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",bhuman resources director expert organizationa...


In [165]:
# get rid of the byte signifier 'b' at beginning
df['resume'] = df['resume'].str.slice_replace(0,1, '') 

In [166]:
df.head()

,ID,Category,dirty_resume,resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",john h smith phr po box callahan fl infog...
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,name surname address mobile noemail personal p...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,anthony brown hr assistant areas expertise per...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,satheesh email id career objective pursue gro...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",human resources director expert organizational...


<br>
<br>
<br>
<br>
<br>
<br>

# Clean Job Posts Data

In [167]:
lightsail_jobs_path = '/home/ec2-user/NLP_projects/job_recommender_project/data/job_posts_madhab.csv'
local_jobs_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/job_posts_madhab.csv'



jobs_path = local_jobs_path

In [168]:
#load and take first look at job data
jobs = pd.read_csv(jobs_path)
jobs.head()

,jobpost,date,Title,Company,AnnouncementCode,Term,Eligibility,Audience,StartDate,Duration,...,Salary,ApplicationP,OpeningDate,Deadline,Notes,AboutC,Attach,Year,Month,IT
0,AMERIA Investment Consulting Company\r\nJOB TI...,"Jan 5, 2004",Chief Financial Officer,AMERIA Investment Consulting Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,"To apply for this position, please submit a\r\...",NaN,26 January 2004,NaN,NaN,NaN,2004,1,False
1,International Research & Exchanges Board (IREX...,"Jan 7, 2004",Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,NaN,NaN,NaN,NaN,3 months,...,NaN,Please submit a cover letter and resume to:\r\...,NaN,12 January 2004,NaN,The International Research & Exchanges Board (...,NaN,2004,1,False
2,Caucasus Environmental NGO Network (CENN)\r\nJ...,"Jan 7, 2004",Country Coordinator,Caucasus Environmental NGO Network (CENN),NaN,NaN,NaN,NaN,NaN,Renewable annual contract\r\nPOSITION,...,NaN,Please send resume or CV toursula.kazarian@......,NaN,20 January 2004\r\nSTART DATE: February 2004,NaN,The Caucasus Environmental NGO Network is a\r\...,NaN,2004,1,False
3,Manoff Group\r\nJOB TITLE: BCC Specialist\r\n...,"Jan 7, 2004",BCC Specialist,Manoff Group,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Please send cover letter and resume to Amy\r\n...,NaN,23 January 2004\r\nSTART DATE: Immediate,NaN,NaN,NaN,2004,1,False
4,Yerevan Brandy Company\r\nJOB TITLE: Software...,"Jan 10, 2004",Software Developer,Yerevan Brandy Company,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Successful candidates should submit\r\n- CV; \...,NaN,"20 January 2004, 18:00",NaN,NaN,NaN,2004,1,True


In [169]:
#far more columns than we need
jobs.columns

Index(['jobpost', 'date', 'Title', 'Company', 'AnnouncementCode', 'Term',
       'Eligibility', 'Audience', 'StartDate', 'Duration', 'Location',
       'JobDescription', 'JobRequirment', 'RequiredQual', 'Salary',
       'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 'Attach',
       'Year', 'Month', 'IT'],
      dtype='object')

In [170]:
#drop columns. Kept Title, Description, Requirement, and Quals
#19001 rows x 5 cols
jobs.drop(['jobpost', 'date', 'AnnouncementCode', 'Term', 'Eligibility',
       'Audience', 'StartDate', 'Duration', 'Location',
       'Salary', 'ApplicationP', 'OpeningDate', 'Deadline', 'Notes', 'AboutC', 
       'Attach', 'Year', 'Month', 'IT'], axis=1, inplace=True)

#renaming 
jobs['JobRequirement'] = jobs['JobRequirment']
jobs.drop(['JobRequirment'], axis=1, inplace=True)
jobs

,Title,Company,JobDescription,RequiredQual,JobRequirement
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...
1,Full-time Community Connections Intern (paid i...,International Research & Exchanges Board (IREX),NaN,- Bachelor's Degree; Master's is preferred;\r\...,NaN
2,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...
3,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...
4,Software Developer,Yerevan Brandy Company,NaN,- University degree; economical background is ...,- Rendering technical assistance to Database M...
...,...,...,...,...,...
18996,Senior Creative UX/ UI Designer,Technolinguistics NGO,A tech startup of Technolinguistics based in N...,- At least 5 years of experience in Interface/...,- Work closely with product and business teams...
18997,Category Development Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,"- University degree, ideally business related;...",- Establish and manage Category Management dev...
18998,Operational Marketing Manager,"""Coca-Cola Hellenic Bottling Company Armenia"" ...",NaN,"- Degree in Business, Marketing or a related f...","- Develop, establish and maintain marketing st..."
18999,Head of Online Sales Department,San Lazzaro LLC,San Lazzaro LLC is looking for a well-experien...,- At least 1 year of experience in online sale...,- Handle the project activites of the online s...


In [171]:
#drop any rows with NaN
#13124 rows x 5 cols from 19001 rows x 5 cols
jobs.dropna(inplace=True)
jobs

,Title,Company,JobDescription,RequiredQual,JobRequirement
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...
2,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...
3,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...
13,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Higher Education and/or professional experie...,- Assist the Tavush Marz communities and commu...
17,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,- Minimum Masters Degree in Economics;\r\n- Mi...,The incumbent under direct supervision of UNDP...
...,...,...,...,...,...
18992,.NET Developer,Macadamian AR CJSC,The incumbent will develop software applicatio...,- 2 - 5 years of experience in software develo...,- Participate in all the steps of the software...
18995,Deputy Director,"""Transport PIU"" State Institution of the RA Mi...",The incumbent will be responsible for supporti...,"- University degree in Civil Engineering, Econ...",- Support the Director in organizing the activ...
18996,Senior Creative UX/ UI Designer,Technolinguistics NGO,A tech startup of Technolinguistics based in N...,- At least 5 years of experience in Interface/...,- Work closely with product and business teams...
18999,Head of Online Sales Department,San Lazzaro LLC,San Lazzaro LLC is looking for a well-experien...,- At least 1 year of experience in online sale...,- Handle the project activites of the online s...


In [172]:
# reset the indexes after dropping NaN values
# give explicit column called 'label' for index
jobs.set_index(np.arange(len(jobs['Title'])), inplace=True)
for i in jobs['Title']:
    jobs['label'] = jobs['Title'].index +1

In [173]:
jobs.head()

,Title,Company,JobDescription,RequiredQual,JobRequirement,label
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...,1
1,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,2
2,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,3
3,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Higher Education and/or professional experie...,- Assist the Tavush Marz communities and commu...,4
4,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,- Minimum Masters Degree in Economics;\r\n- Mi...,The incumbent under direct supervision of UNDP...,5


Descriptions, Requirements, Required Quals

In [174]:
#what does a job description look like
jobs['JobDescription'][0]

"AMERIA Investment Consulting Company is seeking a\r\nChief Financial Officer. This position manages the company's fiscal and\r\nadministrative functions, provides highly responsible and technically\r\ncomplex staff assistance to the Executive Director. The work performed\r\nrequires a high level of technical proficiency in financial management\r\nand investment management, as well as management, supervisory, and\r\nadministrative skills."

In [175]:
#what does a job requirement look like
jobs['JobRequirement'][0]

"- Supervises financial management and administrative staff, including\r\nassigning responsibilities, reviewing employees' work processes and\r\nproducts, counseling employees, giving performance evaluations, and\r\nrecommending disciplinary action;\r\n- Serves as member of management team participating in both strategic\r\nand operational planning for the company;\r\n- Directs and oversees the company's financial management activities,\r\nincluding establishing and monitoring internal controls, managing cash\r\nand investments, and managing the investment portfolio in collaboration\r\nwith the Investment team leader. This includes, but is not limited to,\r\nevaluation of investment risk, concentration risk, fund deployment\r\nlevels, adequacy of loss and liquidity reserves Assists investment team\r\nin development of proper documentation and internal systems;\r\n- Directs and oversees the annual budgeting process, including\r\ndeveloping projections for financial planning, and prepari

In [176]:
#what do the required quals look like?
jobs['RequiredQual'][0]

"To perform this job successfully, an\r\nindividual must be able to perform each essential duty satisfactorily.\r\nThe requirements listed below are representative of the knowledge,\r\nskill, and/or ability required.\r\nKnowledge of:\r\n- Generally accepted accounting principles;\r\n- Local accounting standards and legislation;\r\n- State reporting requirements pertaining to accounting;\r\n- Principles and practices of financial management and budgeting;\r\n- Principles and practices of financial systems design and analysis;\r\n- Principles and practices of contract management, records management,\r\nand risk management;\r\n- Principles and practices of management and supervision;\r\n- Principles and practices of information systems management.\r\nAbility to:\r\n- Apply sound fiscal and administrative practices to the company's\r\nactivities;\r\n- Plan, organize and supervise the work of subordinate employees,\r\nincluding training them, assigning and evaluating their work, and\r\nprov

In [177]:
#make new 'combined' column, concatenate strings from 
# Title, JobDescription, JobRequirement, RequiredQuals
jobs['combined'] = jobs['Title'].str.cat(jobs['JobDescription'], sep=' ').str.cat(jobs['JobRequirement'], sep=' ').str.cat(jobs['RequiredQual'], sep =' ')



In [178]:
jobs['combined'][0]

"Chief Financial Officer AMERIA Investment Consulting Company is seeking a\r\nChief Financial Officer. This position manages the company's fiscal and\r\nadministrative functions, provides highly responsible and technically\r\ncomplex staff assistance to the Executive Director. The work performed\r\nrequires a high level of technical proficiency in financial management\r\nand investment management, as well as management, supervisory, and\r\nadministrative skills. - Supervises financial management and administrative staff, including\r\nassigning responsibilities, reviewing employees' work processes and\r\nproducts, counseling employees, giving performance evaluations, and\r\nrecommending disciplinary action;\r\n- Serves as member of management team participating in both strategic\r\nand operational planning for the company;\r\n- Directs and oversees the company's financial management activities,\r\nincluding establishing and monitoring internal controls, managing cash\r\nand investments,

In [179]:
jobs['dirty_combined'] = jobs['combined']
jobs = full_clean(jobs, 'combined', 'dirty_combined')

In [180]:
jobs.head()

,Title,Company,JobDescription,RequiredQual,JobRequirement,label,combined,dirty_combined
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...,1,chief financial officer ameria investment cons...,Chief Financial Officer AMERIA Investment Cons...
1,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,2,country coordinator public outreach strengthen...,Country Coordinator Public outreach and streng...
2,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,3,bcc specialist lead local enhancement developm...,BCC Specialist The LEAD (Local Enhancement and...
3,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Higher Education and/or professional experie...,- Assist the Tavush Marz communities and commu...,4,community development capacity building confli...,"Community Development, Capacity Building and C..."
4,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,- Minimum Masters Degree in Economics;\r\n- Mi...,The incumbent under direct supervision of UNDP...,5,country economist nob united nations developme...,Country Economist (NOB) The United Nations Dev...


In [181]:
jobs['combined'][0]

'chief financial officer ameria investment consulting company seeking chief financial officer position manages companys fiscal administrative functions provides highly responsible technically complex staff assistance executive director work performed requires high level technical proficiency financial management investment management well management supervisory administrative skills supervises financial management administrative staff including assigning responsibilities reviewing employees work processes products counseling employees giving performance evaluations recommending disciplinary action serves member management team participating strategic operational planning company directs oversees companys financial management activities including establishing monitoring internal controls managing cash investments managing investment portfolio collaboration investment team leader includes limited evaluation investment risk concentration risk fund deployment levels adequacy loss liquidity

<br>
<br>
<br>
<br>
<br>

## Save cleaned data to CSV

In [182]:
df.head()

,ID,Category,dirty_resume,resume
0,1,HR,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box...",john h smith phr po box callahan fl infog...
1,2,HR,b'Name Surname\nAddress\nMobile No/Email\nPERS...,name surname address mobile noemail personal p...
2,3,HR,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...,anthony brown hr assistant areas expertise per...
3,4,HR,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...,satheesh email id career objective pursue gro...
4,5,HR,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert...",human resources director expert organizational...


In [183]:
jobs.head()

,Title,Company,JobDescription,RequiredQual,JobRequirement,label,combined,dirty_combined
0,Chief Financial Officer,AMERIA Investment Consulting Company,AMERIA Investment Consulting Company is seekin...,"To perform this job successfully, an\r\nindivi...",- Supervises financial management and administ...,1,chief financial officer ameria investment cons...,Chief Financial Officer AMERIA Investment Cons...
1,Country Coordinator,Caucasus Environmental NGO Network (CENN),Public outreach and strengthening of a growing...,"- Degree in environmentally related field, or ...",- Working with the Country Director to provide...,2,country coordinator public outreach strengthen...,Country Coordinator Public outreach and streng...
2,BCC Specialist,Manoff Group,The LEAD (Local Enhancement and Development fo...,"- Advanced degree in public health, social sci...",- Identify gaps in knowledge and overseeing in...,3,bcc specialist lead local enhancement developm...,BCC Specialist The LEAD (Local Enhancement and...
3,"Community Development, Capacity Building and C...",Food Security Regional Cooperation and Stabili...,Food Security Regional Cooperation and Stabili...,- Higher Education and/or professional experie...,- Assist the Tavush Marz communities and commu...,4,community development capacity building confli...,"Community Development, Capacity Building and C..."
4,Country Economist (NOB),"United Nations Development Programme, Armenia",The United Nations Development Programme in Ar...,- Minimum Masters Degree in Economics;\r\n- Mi...,The incumbent under direct supervision of UNDP...,5,country economist nob united nations developme...,Country Economist (NOB) The United Nations Dev...


In [184]:
# save resumes to csv
lightsail_resume_cleaned_path = r'/home/ec2-user/NLP_projects/job_recommender_project/data/cleaned_resume_dataset_maitrip.csv'
local_resume_cleaned_path = r'/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_resume_dataset_maitrip.csv'

df.to_csv(local_resume_cleaned_path, index=False)


In [185]:
# save jobs to csv
lightsail_jobs_cleaned_path = '/home/ec2-user/NLP_projects/job_recommender_project/data/cleaned_job_posts_madhab.csv'
local_jobs_cleaned_path = '/Users/richardkuzma/coding/NLP_projects/job_recommender_project/data/cleaned_job_posts_madhab.csv'

jobs.to_csv(local_jobs_cleaned_path, index=False)


In [186]:
df['resume'][2]

'anthony brown hr assistant areas expertise personal summary hr processes systems competent organised individual able work part team manage several priorities one time anthony positive attitude strong work ethic keen desire learn grow within firm possesses superb communications skills always treats people respect according individual needs dedicated professional fully understands importance hr department organisation therefore aims make office works effective efficient possible extensive experience working commercially focussed organisations fully understands pressures achieving targets accurately assessing job applicants according ability contract document generation accepting resignations business administration note taking right would like work friendly exciting company looking hr assistant reflect values excellence quality recruitment methodologies career history employment legislation answering queries document management equal opportunities absence management calendar management 

In [187]:
df['dirty_resume'][2]

"b'Anthony Brown\\nHR Assistant\\nAREAS OF EXPERTISE\\n\\nPERSONAL SUMMARY\\n\\nHR processes & systems\\n\\nA competent and organised individual who is able to work as part of a team and\\nmanage several priorities at any one time. Anthony has a positive attitude,\\nstrong work ethic, and a keen desire to learn and grow within a firm. He\\npossesses superb communications skills, and always treats people with respect\\nand according to their individual needs. As a dedicated professional he fully\\nunderstands the importance of the HR department to any organisation, and\\ntherefore aims to make any office he works in as effective and efficient as\\npossible. He has extensive experience of working in commercially focussed\\norganisations, and fully understands the pressures of achieving targets and\\naccurately assessing job applicants according to their ability.\\n\\nContract document\\ngeneration\\nAccepting resignations\\nBusiness Administration\\nNote taking\\n\\nRight now he would li